# Purpose

This script is used to apply the 9 timseries cross-validation models to the the test data (for 2023). 

In [ ]:
#high level modules
import os
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# custom modules
this_dir = "/Users/steeleb/Documents/GitHub/NASA-NW/modeling/temperature/arNN_fewer2/"

imp.load_source("universals", os.path.join(this_dir, "universal_functions.py"))
from universals import load_pickle_file, twotemp_labels_features_test, predict_2_values_test, return_test_error_metrics

imp.load_source("vis", os.path.join(this_dir, "vis_functions.py"))
from vis import ts_plot, create_scatter_plot_test

We need to read in the transformed test data, apply the models, and save the resictions.

In [3]:
# import test data
file_path = os.path.expanduser("/Users/steeleb/Documents/GitHub/NASA-NW/data/NN_train_val_test/SMR_autoNN_reduce_2/")

test_fn = os.path.join(file_path, "t2023_reduced_standardized_v2024-06-03.csv")
with open(test_fn) as f:
    test = pd.read_csv(f, sep=',')

And we need to filter this for our target date - Jul 1 through Sept 11.

In [4]:
# arrange the data by date
test['date'] = pd.to_datetime(test['date'])
test = test.sort_values(by='date')
test = test.loc[test['date'].between('2023-07-01', '2023-09-11')]

Now we need to apply the models to the test data, but first we need to format the data for the models

In [ ]:
test_features, test_labels = twotemp_labels_features_test(test)
test_features.shape, test_labels.shape

Load the models

In [6]:
# load the models
model_dir = '/Users/steeleb/Documents/GitHub/NASA-NW/data/NN_train_val_test/SMR_autoNN_reduce_2/models/leaky_basic_5_t2023/'

models = [f for f in os.listdir(model_dir) if 'history' not in f]

model_1 = load_pickle_file(models[0], model_dir)
model_2 = load_pickle_file(models[1], model_dir)
model_3 = load_pickle_file(models[2], model_dir)
model_4 = load_pickle_file(models[3], model_dir)
model_5 = load_pickle_file(models[4], model_dir)
model_6 = load_pickle_file(models[5], model_dir)
model_7 = load_pickle_file(models[6], model_dir)
model_8 = load_pickle_file(models[7], model_dir)
model_9 = load_pickle_file(models[8], model_dir)


Now we need to apply the models to the test data and get the resictions, for which we also need the standarization parameters to back calculate.

In [ ]:
transform = pd.read_csv(os.path.join("/Users/steeleb/Documents/GitHub/NASA-NW/data/NN_train_val_test/SMR_autoNN_reduce_2/", "mean_std_train_val_t2023_v2024-05-21.csv"), sep=',')
transform = transform.rename(columns={"Unnamed: 0": "feature"}).set_index("feature")

t_mean_1m = transform['mean'].get('mean_1m_temp_degC')
t_std_1m = transform['std'].get('mean_1m_temp_degC')

t_mean_05m = transform['mean'].get('mean_0_5m_temp_degC')
t_std_05m = transform['std'].get('mean_0_5m_temp_degC')

test["pred_1m_1"], test["pred_05m_1"] = predict_2_values_test(model_1, test_features, t_mean_1m, t_mean_05m, t_std_1m, t_std_05m)
test["pred_1m_2"], test["pred_05m_2"] = predict_2_values_test(model_2, test_features, t_mean_1m, t_mean_05m, t_std_1m, t_std_05m)
test["pred_1m_3"], test["pred_05m_3"] = predict_2_values_test(model_3, test_features, t_mean_1m, t_mean_05m, t_std_1m, t_std_05m)
test["pred_1m_4"], test["pred_05m_4"] = predict_2_values_test(model_4, test_features, t_mean_1m, t_mean_05m, t_std_1m, t_std_05m)
test["pred_1m_5"], test["pred_05m_5"] = predict_2_values_test(model_5, test_features, t_mean_1m, t_mean_05m, t_std_1m, t_std_05m)
test["pred_1m_6"], test["pred_05m_6"] = predict_2_values_test(model_6, test_features, t_mean_1m, t_mean_05m, t_std_1m, t_std_05m)
test["pred_1m_7"], test["pred_05m_7"] = predict_2_values_test(model_7, test_features, t_mean_1m, t_mean_05m, t_std_1m, t_std_05m)
test["pred_1m_8"], test["pred_05m_8"] = predict_2_values_test(model_8, test_features, t_mean_1m, t_mean_05m, t_std_1m, t_std_05m)
test["pred_1m_9"], test["pred_05m_9"] = predict_2_values_test(model_9, test_features, t_mean_1m, t_mean_05m, t_std_1m, t_std_05m)

And now, let's make the ensemble prediction to assess the model performance.

In [8]:
test["ts_cv_ensemble_pred_1m"] = np.mean(test[['pred_1m_1', 'pred_1m_2', 'pred_1m_3', 'pred_1m_4', 'pred_1m_5', 'pred_1m_6', 'pred_1m_7', 'pred_1m_8']], axis=1)
test["ts_cv_ensemble_pred_05m"] = np.mean(test[['pred_05m_1', 'pred_05m_2', 'pred_05m_3', 'pred_05m_4', 'pred_05m_5', 'pred_05m_6', 'pred_05m_7', 'pred_05m_8']], axis=1)

In order to compare these with the actual values, we need to load the actual values and calculate the performance metrics. To do this, we will back-calculate using the mean and standard deviation.

In [9]:
test['mean_1m_temp_degC'] = test['mean_1m_temp_degC'] * t_std_1m + t_mean_1m
test['mean_0_5m_temp_degC'] = test['mean_0_5m_temp_degC'] * t_std_05m + t_mean_05m

Let's look at the data from 1m first:

In [ ]:
return_test_error_metrics(test["mean_1m_temp_degC"], test["ts_cv_ensemble_pred_1m"])

And then the data from 0-5m:

In [ ]:
return_test_error_metrics(test["mean_0_5m_temp_degC"], test["ts_cv_ensemble_pred_05m"])

Let's look at this in context of the data:

In [ ]:
create_scatter_plot_test("1m Temp Ensemble", test["ts_cv_ensemble_pred_1m"], test["mean_1m_temp_degC"])
create_scatter_plot_test("0-05m Temp Ensemble", test["ts_cv_ensemble_pred_05m"], test["mean_0_5m_temp_degC"])

In [ ]:
ts_plot(test["mean_1m_temp_degC"], test["ts_cv_ensemble_pred_1m"], "1m Temp Ensemble")
ts_plot(test["mean_0_5m_temp_degC"], test["ts_cv_ensemble_pred_05m"], "0-5m Temp Ensemble")


In [ ]:
plt.figure(figsize=(10,4))
plt.plot(test['date'], test["pred_1m_1"], label="Individual Models", color='grey')
plt.plot(test['date'], test["pred_1m_2"], color='grey')
plt.plot(test['date'], test["pred_1m_3"], color='grey')
plt.plot(test['date'], test["pred_1m_4"], color='grey')
plt.plot(test['date'], test["pred_1m_5"], color='grey')
plt.plot(test['date'], test["pred_1m_6"], color='grey')
plt.plot(test['date'], test["pred_1m_7"], color='grey')
plt.plot(test['date'], test["pred_1m_8"], color='grey')
plt.plot(test['date'], test["pred_1m_9"], color='grey')
plt.plot(test['date'], test["ts_cv_ensemble_pred_1m"], label="Ensemble Mean", color='blue')
plt.plot(test['date'], test["mean_1m_temp_degC"], label="Observed", color='red')
plt.xlabel("date")
plt.ylabel("Temperature ($^\circ$C)")
plt.legend()
plt.title("All models and ensemble mean for 1m temperature")
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(test['date'], test["pred_05m_1"], label="Individual Models", color='grey')
plt.plot(test['date'], test["pred_05m_2"], color='grey')
plt.plot(test['date'], test["pred_05m_3"], color='grey')
plt.plot(test['date'], test["pred_05m_4"], color='grey')
plt.plot(test['date'], test["pred_05m_5"], color='grey')
plt.plot(test['date'], test["pred_05m_6"], color='grey')
plt.plot(test['date'], test["pred_05m_7"], color='grey')
plt.plot(test['date'], test["pred_05m_8"], color='grey')
plt.plot(test['date'], test["pred_05m_9"], color='grey')
plt.plot(test['date'], test["ts_cv_ensemble_pred_05m"], label="Ensemble Mean", color='blue')
plt.plot(test['date'], test["mean_0_5m_temp_degC"], label="Observed", color='red')
plt.xlabel("date")
plt.ylabel("Temperature ($^\circ$C)")
plt.legend()
plt.title("All models and ensemble mean for 0-5m temperature")
plt.show()

These actually look quite good. While the model doesn't capture the highest values at 1m, there were zero instances in the training/validation set with values that high, so it's not surprising that the model doesn't capture them. The model does a good job of capturing the general trends in the data and does very well with the high 0-5m values.

Let's look at the error metrics for the models:

1m results:

Mean Squared Error: 0.32

Mean Absolute Error: 0.48

Root Mean Squared Error: 0.56

Mean Absolute Percentage Error: 0.03


0-5m results:

Mean Squared Error: 0.10

Mean Absolute Error: 0.25

Root Mean Squared Error: 0.32

Mean Absolute Percentage Error: 0.02


Baseline:

1m baseline

Mean Squared Error for 1m baseline : 0.24

Mean Absolute Error for 1m baseline : 0.40

0-5m baseline:

Mean Squared Error for 0-5m baseline : 0.11

Mean Absolute Error for 0-5m baseline : 0.28

## Create timeseries residual plot

Calculate residuals

In [16]:
test['res_1m_1'] = test['pred_1m_1'] - test['mean_1m_temp_degC']
test['res_1m_2'] = test['pred_1m_2'] - test['mean_1m_temp_degC']
test['res_1m_3'] = test['pred_1m_3'] - test['mean_1m_temp_degC']
test['res_1m_4'] = test['pred_1m_4'] - test['mean_1m_temp_degC']
test['res_1m_5'] = test['pred_1m_5'] - test['mean_1m_temp_degC']
test['res_1m_6'] = test['pred_1m_6'] - test['mean_1m_temp_degC']
test['res_1m_7'] = test['pred_1m_7'] - test['mean_1m_temp_degC']
test['res_1m_8'] = test['pred_1m_8'] - test['mean_1m_temp_degC']
test['res_1m_9'] = test['pred_1m_9'] - test['mean_1m_temp_degC']
test['res_1m_ensemble'] = test['ts_cv_ensemble_pred_1m'] - test['mean_1m_temp_degC']

test['res_05m_1'] = test['pred_05m_1'] - test['mean_0_5m_temp_degC']
test['res_05m_2'] = test['pred_05m_2'] - test['mean_0_5m_temp_degC']
test['res_05m_3'] = test['pred_05m_3'] - test['mean_0_5m_temp_degC']
test['res_05m_4'] = test['pred_05m_4'] - test['mean_0_5m_temp_degC']
test['res_05m_5'] = test['pred_05m_5'] - test['mean_0_5m_temp_degC']
test['res_05m_6'] = test['pred_05m_6'] - test['mean_0_5m_temp_degC']
test['res_05m_7'] = test['pred_05m_7'] - test['mean_0_5m_temp_degC']
test['res_05m_8'] = test['pred_05m_8'] - test['mean_0_5m_temp_degC']
test['res_05m_9'] = test['pred_05m_9'] - test['mean_0_5m_temp_degC']
test['res_05m_ensemble'] = test['ts_cv_ensemble_pred_05m'] - test['mean_0_5m_temp_degC']


In [ ]:
yit = test.filter(['mean_0_5m_temp_degC', 'mean_1m_temp_degC'])
yit = yit.shift(periods=1).rename(columns={'mean_0_5m_temp_degC': 'yit_mean_0_5m_temp_degC', 
                                     'mean_1m_temp_degC': 'yit_mean_1m_temp_degC'})
print(yit)
test = pd.concat([test, yit], axis=1)
test['res_yit_0_5m'] = test['yit_mean_0_5m_temp_degC'] - test['mean_0_5m_temp_degC']
test['res_yit_1m'] = test['yit_mean_1m_temp_degC'] - test['mean_1m_temp_degC']


In [ ]:
plt.figure(figsize=(10,4))
plt.axhline(0, color = 'black')
plt.plot(test['date'], test["res_05m_1"], label="Individual models", color='grey')
plt.plot(test['date'], test["res_05m_2"], color='grey')
plt.plot(test['date'], test["res_05m_3"], color='grey')
plt.plot(test['date'], test["res_05m_4"], color='grey')
plt.plot(test['date'], test["res_05m_5"], color='grey')
plt.plot(test['date'], test["res_05m_6"], color='grey')
plt.plot(test['date'], test["res_05m_7"], color='grey')
plt.plot(test['date'], test["res_05m_8"], color='grey')
plt.plot(test['date'], test["res_05m_9"], color='grey')
plt.plot(test['date'], test["res_05m_ensemble"], label="Ensemble mean", color='red')
#plt.plot(test['date'], test["res_yit_0_5m"], label = "yesterday-is-today", color = 'blue')
plt.ylabel("Model Residual ($^\circ$C)")
plt.legend(ncols = 3)
plt.title("Residuals for 0-5m temperature")
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
plt.axhline(0, color = 'black')
plt.plot(test['date'], test["res_1m_1"], label="Individual models", color='grey')
plt.plot(test['date'], test["res_1m_2"], color='grey')
plt.plot(test['date'], test["res_1m_3"], color='grey')
plt.plot(test['date'], test["res_1m_4"], color='grey')
plt.plot(test['date'], test["res_1m_5"], color='grey')
plt.plot(test['date'], test["res_1m_6"], color='grey')
plt.plot(test['date'], test["res_1m_7"], color='grey')
plt.plot(test['date'], test["res_1m_8"], color='grey')
plt.plot(test['date'], test["res_1m_9"], color='grey')
plt.plot(test['date'], test["res_1m_ensemble"], label="Ensemble mean", color='red')
#plt.plot(test['date'], test["res_yit_1m"], label = "yesterday-is-today", color = 'blue')
plt.ylabel("Model Residual ($^\circ$C)")
plt.legend(ncols = 3)
plt.title("Residuals for 1m temperature")
plt.show()